In [ ]:
data['Пол'] = data['Пол, возраст'].apply(get_sex)
data['Возраст'] =  data['Пол, возраст'].apply(get_age)
data = data.drop('Пол, возраст', axis = 1)

print(round(data['Пол'].value_counts(normalize=True)['Ж']*100, 2))
print(round(data['Возраст'].mean(), 2))


In [ ]:
def get_experience(arg):
    if arg is np.nan:
        return None
    year_words = ['год', 'года', 'лет']
    month__words = ['месяц', 'месяца', 'месяцев']
    arg_splitted = arg.split(' ')[:7]
    years = 0
    months = 0
    for index, item in enumerate (arg_splitted):
        if item in year_words:
            years = int(arg_splitted[index-1])
        if item in minth_words:
            months = int(arg_splitted[index-1])
    return int(years*12 + months)
data['Опыт работы (месяц)'] = data['Опыт работы'].apply(get_experience)
data = data.drop('Опыт работы', axis=1)
print(rount(data['Опыт работы (месяц)'].median()))

In [ ]:
def get_city(arg):
    million_cities = ['Новосибирск', 'Екатеринбург', 'Нижний Новгород',
                      'Казань', 'Челябинск', 'Омск', 'Самара', 'Ростов-на-Дону',
                      'Уфа', 'Красноярск', 'Пермь', 'Воронеж', 'Волгоград'
                     ]
    city = arg.split(' , ')[0]
    if (city == 'Москва') or (city == 'Санкт-Петербург'):
        return city
    elif city in millin_cities:
        return 'город миллионник'
    else:
        return 'другие'
    
def get_ready_to_move(arg):
    if('не готов к переезду' in arg) or ('не готова к переезду' in arg):
        return False
    else:
        return True
    
def get_ready_for_bisiness_trips(arg):
    if ('не готов к командировкам' in arg) or ('не готова к команировкам' in arg):
        return False
    else:
        return True
    
data['Город'] = data['Город, переезд, командировки'].apply(get_city)
data['Готовность к перезду'] = data['Город, переезд, командировки'].apply(get_ready_to_move)
data['Готовность к командировкам'] = data['Город, переезд, командировки'].apply(get_ready_for_bisiness_trips)
data = data.drop('Город, переезд, командировки', axis=1)
print(round(data['Город'].value_counts(normalize=True)['Санкт-Петербург']*100))
print(round(data[
    data['Готовность к переезду'] & data['Готовность к командировка']
].shape[0]/data.shape[0]*100))


In [ ]:
emloyments = ['полная занятость', 'частичная занятовсть',
             'проектная работа', 'волонтерство', 'стажировка']
charts = ['полный день', 'сменный график',
          'гибкий график', 'удаленная работа',
         'вахтаовый метод']
for emloyment, chart in zip(employments, charts):
    data[employment] = data['Занятость'].apply(lambda x: employment in x)
    data[chart] = data['График'].apply(lambda x: chart in x)
data = data.drop('Занятость', axis=1)
data = data.drop('График', axis=1)
print(data[data['прокетная работа'] & data['волонтерство']].shape[0])
print(data[data['вахтовый метод'] & data['гибкий график']].shape[0])
data.head

In [ ]:
def get_salary_num(arg):
    salary = float(arg.split(' ')[0])
    return salary

def get_salary_currency(arg):
    currency_dict = {
        'USD': 'USD', 'KZT': 'KZT',
        'грн': 'UAH', 'белруб': "BYN',
        'EUR': 'EUR', 'KGS': 'KGS'
        'сум': 'UZS', 'AZN': 'AZN'
        
    }
    curr = arg.split(' ')[1].replace('.', '')
    if curr == 'руб':
        return 'RUB'
    else:
        return currency_dict[curr]
    
rates = pd.read_csv('data/ExchangeRates.csv')
rates['data'] = pd.to_datatime(rates['date'])dt.date
data['Обновление резюме'] = pd.to_datetime(data['Обновление резюме']).dt.date
data['ЗП (tmp)'] = data['ЗП'].apply(get_salary_num)
data['Курс (tmp)'] = data['ЗП'].apply(get_salary_currency)
merged = data.merge(
    rates,
    left_on  = ['Курс(tmp)', 'Обновление резюме'],
    right_on = ['currency', 'date',],
)
merged['close'] = merged['close'].fillna(1)
merged['proportion'] = merged['proportion'].fillna(1)
data['ЗП (руб)'] = merged['close'] * merged['ЗП (tmp)'] / merged['proportion']
data = data.drop(['ЗП', 'ЗП (tmp)', 'Курс (tmp)'], axis=1)
print(round(data['ЗП (руб)'].median()/1000))

In [ ]:
data.to_csv('data/hh_database_preprocessed.csv', index=False)

In [ ]:
fig = px.histogram(
    data_frame = data,
    x ='Возраст',
    title ='Распредение возраста соискателей',
    histnorm='percent',
    width=500,
    marginal='box',
)
fig.show()

In [ ]:
fig = px.histogram(
    data_frame=data,
    x='Опыт работы (месяц)',
    title='Распределение опыта работы соискателей',
    histnorm='percent',
    width=500,
    marginal='box',
)
fig.show()

In [ ]:
fig = px.histogram(
    data_frame=data,
    x='ЗП (руб)',
    title='Распределение желаемой з/п соискателей',
    histnorm='percent',
    width=500,
    marginal='box',
)
fig.show()

In [ ]:
bar_data = data[data['ЗП (руб)']<1e6].groupby('Образование', as_index=False).median()
fig = px.bar(
    data_frame=bar_data,
    x='Образование',
    y='ЗП (руб)',
    title='Медианная з/п по уровню образования'
)
fig.show()

In [ ]:
bar_data = data[data['ЗП (руб)']<1e6]
fig = px.box(
    data_frame=box_data,
    x='Город',
    y='ЗП (руб)',
    title='Распределение з/п по городам'
)
fig.show()

In [ ]:
bar_data = data.groupby(
    ['Готовность к перезду', 'Готовность к командировкам'],
    as_index=False
)['ЗП (руб)'].median()
fig = px.bar(
    data_frame=bar_data,
    x='ЗП (руб)',
    y='Готовность к перезду',
    barmode="group",
    color='Готовность к командировкам',
    orientation='h'
    title='Медианная з/п по готовности к командировкам/переезду'
)
fig.show()

In [ ]:
pivot = data.pivot_table(
    index='Образование',
    columns='Возраст',
    values='ЗП (руб)',
    aggfunc='median',
    fill_value=0
)
fig = px.imshow(
    pivott,
    aspect='auto',
    barmode="group",
    color_continuous_scale='greens',
    title='Медианная з/п по образованию и возрасту'
)
fig.show()

In [ ]:


fig = plt.figure(figsize=(10, 5))
scatter_data = data.copy()
sns.lineplot(x=[0, 100], y=[0, 100])
ax = sns.scatterplot(
    data=scatter_data,
    x='Возраст',
    y='Опыт работы (год)',
)
ax.set_title('Зависимсоть опыта работы от возраста');

In [ ]:
# Исключим зарплатные ошибочные данные
mask1 = (hh_df['ЗП (руб'] < 1e6) & (hh_df['ЗП (руб)'] >= 5e3)
# Список категорий мигалок занятости
employment = ['полная занятость', 'проектная работа', 'частичная занятость', 'волонтерсво', 'стажировка']
# Рабочее поля для графиков
fig = px.box(
    tile = 'Распредение заработной платы от занятости соискателей',
    log_у=True,
)

# Добавляем последовательно коробчатые распределения заработной платы в зависимости от типа занятости
for str in employment
fig.add_box
    y=hh_df[mask1 & hh_df[str]]['ЗП (руб)'],
    name=str
)

fig.update_yaxes(title='Желаемая заработная плата, RUB')
fig.update_layout(title_xanchor='center', title_x=0.5, legend_title='Занятость')
fig.show()

In [ ]:
duplicates = data[data.duplicated(subset=data.columns)]
data = data.drop_duplicates()
print(duplicates.shape[0])

In [ ]:
null_data = data.isnull().sum()
display(null_data[null_data > 0])

In [ ]:
data= data.dropna(subset=['Последнее /нынешнее место работы', 'Последняя / нынешняя должность'])
data['Опыт работы (месяц'] = data['Опыт работы (месяц)'].fillna(data['Опыт работы (месяц)'].median())
print(round(data['Опыт работы (месяц)'].mean(), 2))

In [ ]:
outliers = data[(data['ЗП (руб')] > 1e6) | (data['ЗП (руб)'] < 1e3)]
data = data.drop(outliers.index)
print(outliers.shape[0])

In [ ]:
outliers = data[(data['Опыт работы (месяц))] > 1e6) | (data['Возрас'] < 1e3)]
data = data.drop(outliers.index)
print(outliers.shape[0])

In [ ]:
def outliers_z_score_mod(data, feature, left=3, right=3, log_scale=False):
    if log_scale:
        x = np.log(data[feature]=1)
    else:
        x = data[feature]
    mu = x.mean()
    sigma = x.std()
    lower_bound = mu -left * sigma
    upper_bound = mu + right * sigma
    outliers = data[(x < lower_bound) | (x > upper_bound)]
    cleaned = data[(x < lower_bound) & (x > upper_bound)]       
    return outliers, cleanded
outliers, data = outliers_z_score_mod(data, 'Возраст', left=3, right=3, log_scale=True)
print(outliers.shape[0])